In [2]:
from google.colab import files
uploaded = files.upload()


Saving test.csv to test.csv
Saving train.csv to train.csv
Saving val.csv to val.csv


验证是否使用gpu训练

In [1]:
import torch
print("GPU 是否可用:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU 名称:", torch.cuda.get_device_name(0))


GPU 是否可用: True
GPU 名称: Tesla T4


In [5]:

!pip install transformers datasets scikit-learn

import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score

# === 1. 加载 CSV 数据集并转为 Dataset === #
train_df = pd.read_csv("/content/train.csv")
val_df = pd.read_csv("/content/val.csv")
test_df = pd.read_csv("/content/test.csv")

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(val_df),
    "test": Dataset.from_pandas(test_df)
})

# === 2. 分词器预处理 === #
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# === 3. 加载模型 === #
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

# === 4. 数据填充器 === #
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# === 5. 评估函数 === #
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='macro')
    return {"accuracy": acc, "macro_f1": f1}

# === 6. 训练参数设置 === #
training_args = TrainingArguments(
    output_dir="./results",
    do_eval=True,
    save_strategy="epoch",
    # Changed evaluation_strategy to "eval_strategy" which is the standard in older transformers versions
    eval_strategy="epoch", # Corrected argument name
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1"
)

# === 7. Trainer 定义 === #
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# === 8. 训练模型 === #
trainer.train()

# === 9. 评估测试集 === #
metrics = trainer.evaluate(tokenized_datasets["test"])
print("Test metrics:", metrics)

# === 10. 保存模型（可选） === #
trainer.save_model("./bert-financial-sentiment")

Map:   0%|          | 0/1584 [00:00<?, ? examples/s]

Map:   0%|          | 0/340 [00:00<?, ? examples/s]

Map:   0%|          | 0/340 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-5-ca740dec811f>:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lx221501016 (lx221501016-nanjing-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,0.459100,0.311745,0.867647,0.776819
2,0.159400,0.165952,0.961765,0.949285
3,0.082300,0.112088,0.964706,0.954497


Test metrics: {'eval_loss': 0.15318280458450317, 'eval_accuracy': 0.9470588235294117, 'eval_macro_f1': 0.9248184936874746, 'eval_runtime': 1.4612, 'eval_samples_per_second': 232.683, 'eval_steps_per_second': 15.056, 'epoch': 3.0}


In [4]:
#ALBERT代替Bert
!pip install transformers datasets scikit-learn

import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score

# === 1. 加载 CSV 数据集并转为 Dataset === #
train_df = pd.read_csv("/content/train.csv")
val_df = pd.read_csv("/content/val.csv")
test_df = pd.read_csv("/content/test.csv")

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(val_df),
    "test": Dataset.from_pandas(test_df)
})

# === 2. 分词器预处理 === #
# Note: You define tokenizer twice here. The second definition using albert-base-v2
# is the one that will be used for the model.
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding=True)

# Add the tokenization step here
tokenized_datasets = dataset.map(tokenize_function, batched=True)


tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=3)

# === 4. 数据填充器 === #
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# === 5. 评估函数 === #
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='macro')
    return {"accuracy": acc, "macro_f1": f1}

# === 6. 训练参数设置 === #
training_args = TrainingArguments(
    output_dir="./results",
    do_eval=True,
    save_strategy="epoch",
    eval_strategy="epoch", # Corrected argument name
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1"
)

# === 7. Trainer 定义 === #
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# === 8. 训练模型 === #
trainer.train()

# === 9. 评估测试集 === #
metrics = trainer.evaluate(tokenized_datasets["test"])
print("Test metrics:", metrics)

# === 10. 保存模型（可选） === #
trainer.save_model("./bert-financial-sentiment")

Map:   0%|          | 0/1584 [00:00<?, ? examples/s]

Map:   0%|          | 0/340 [00:00<?, ? examples/s]

Map:   0%|          | 0/340 [00:00<?, ? examples/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-4-e289405b8fed>:66: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lx221501016 (lx221501016-nanjing-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,0.908700,0.799883,0.688235,0.411729
2,0.728400,0.685180,0.711765,0.477962
3,0.587100,0.667459,0.723529,0.480873


Test metrics: {'eval_loss': 0.5760584473609924, 'eval_accuracy': 0.7676470588235295, 'eval_macro_f1': 0.5190264379535329, 'eval_runtime': 1.6319, 'eval_samples_per_second': 208.352, 'eval_steps_per_second': 13.482, 'epoch': 3.0}
